In [1]:
import pandas as pd
from google.cloud import bigquery

# Getting `'appln_id'` with unique `'prior_appln_id'`

In [16]:
client = bigquery.Client()

# Creating Job Config
job_config = bigquery.QueryJobConfig()
#job_config.dry_run = True
job_config.use_query_cache = False
# Set configuration.query.writeDisposition
job_config.write_disposition = 'WRITE_TRUNCATE'

# Set the destination table
dataset_id = 'results_NA'
table_id = 'UniquePF'
table_ref = client.dataset(dataset_id).table(table_id)
job_config.destination = table_ref


query = """
    SELECT *
    FROM (
            SELECT *
            FROM 
                    `usptobias.patstat.tls204` AS t1

            INNER JOIN (
                    SELECT
                            appln_id,
                            COUNT(prior_appln_id) AS nb_PF
                    FROM
                            `usptobias.patstat.tls204` AS t1
                    GROUP BY
                            appln_id
                    HAVING 
                            nb_PF<2
                 ) AS t2
                 USING(appln_id)    
        ) AS t1
  
    LEFT JOIN (
            SELECT
                    appln_id,
                    appln_auth
            FROM
                    `usptobias.patstat.tls201`
            ) AS t2
            USING(appln_id)
     """
# Defining the query
query_job = client.query(query, location='US', job_config=job_config)

query_job.result()

In [17]:
# from google.cloud import bigquery
# client = bigquery.Client()
dataset_id = 'results_NA'
table_id = 'UniquePF'

dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)
table = client.get_table(table_ref)  # API Request

# View table properties
print(table.schema)
print(table.description)
print("\nThere are {:,} number of records in total!".format(table.num_rows))

# Load the first 10 rows
rows = client.list_rows(table, max_results=10000)

#pd.DataFrame(rows)
df_ = rows.to_dataframe()

[SchemaField('appln_id', 'INTEGER', 'NULLABLE', None, ()), SchemaField('prior_appln_id', 'INTEGER', 'NULLABLE', None, ()), SchemaField('prior_appln_seq_nr', 'INTEGER', 'NULLABLE', None, ()), SchemaField('nb_PF', 'INTEGER', 'NULLABLE', None, ()), SchemaField('appln_auth', 'STRING', 'NULLABLE', None, ())]
None

There are 27,558,123 number of records in total!


In [18]:
df_.head()

,appln_id,prior_appln_id,prior_appln_seq_nr,nb_PF,appln_auth
0,23750084,622945,1,1,GR
1,43643352,22819446,1,1,SG
2,340583241,55900685,1,1,MA
3,491415,21933861,1,1,AR
4,42297442,10020815,1,1,PL


# Filtering based on number of members per each (prior_appln_id, appln_auth) group

In [42]:
client = bigquery.Client()

# Creating Job Config
job_config = bigquery.QueryJobConfig()
#job_config.dry_run = True
job_config.use_query_cache = False
# Set configuration.query.writeDisposition
job_config.write_disposition = 'WRITE_TRUNCATE'

# Set the destination table
dataset_id = 'results_NA'
table_id = 'UniquePauth'
table_ref = client.dataset(dataset_id).table(table_id)
job_config.destination = table_ref


query = """
    SELECT *
    FROM
            `usptobias.results_NA.UniquePF`
    LEFT JOIN (
                    SELECT 
                            prior_appln_id,
                            appln_auth,
                            COUNT(appln_id) AS size_PAuth

                    FROM
                            `usptobias.results_NA.UniquePF`
                    GROUP BY
                            prior_appln_id, appln_auth
                    HAVING
                            size_PAuth<2
            )
            USING(prior_appln_id, appln_auth)
    WHERE
            size_PAuth<2

     """
# Defining the query
query_job = client.query(query, location='US', job_config=job_config)

query_job.result()

In [43]:
# from google.cloud import bigquery
# client = bigquery.Client()
dataset_id = 'results_NA'
table_id = 'UniquePauth'

dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)
table = client.get_table(table_ref)  # API Request

# View table properties
print(table.schema)
print(table.description)
print("\nThere are {:,} number of records in total!".format(table.num_rows))

# Load the first 10 rows
rows = client.list_rows(table, max_results=10000)

#pd.DataFrame(rows)
df_ = rows.to_dataframe()

[SchemaField('prior_appln_id', 'INTEGER', 'NULLABLE', None, ()), SchemaField('appln_auth', 'STRING', 'NULLABLE', None, ()), SchemaField('appln_id', 'INTEGER', 'NULLABLE', None, ()), SchemaField('prior_appln_seq_nr', 'INTEGER', 'NULLABLE', None, ()), SchemaField('nb_PF', 'INTEGER', 'NULLABLE', None, ()), SchemaField('size_PAuth', 'INTEGER', 'NULLABLE', None, ())]
None

There are 24,449,360 number of records in total!


In [44]:
df_.head()

,prior_appln_id,appln_auth,appln_id,prior_appln_seq_nr,nb_PF,size_PAuth
0,12915657,EE,15578068,1,1,1
1,905639864,IL,24249561,1,1,1
2,906165127,IL,24246580,1,1,1
3,20355312,IN,24365013,1,1,1
4,43555610,SK,43707862,1,1,1


In [45]:
client = bigquery.Client()

# Creating Job Config
job_config = bigquery.QueryJobConfig()
#job_config.dry_run = True
job_config.use_query_cache = False
# Set configuration.query.writeDisposition
job_config.write_disposition = 'WRITE_TRUNCATE'

# Set the destination table
dataset_id = 'results_NA'
table_id = 'UniquePauth2'
table_ref = client.dataset(dataset_id).table(table_id)
job_config.destination = table_ref


query = """
    SELECT
            *
    FROM (
            SELECT
                    appln_id,
                    prior_appln_id AS custom_family_id
            FROM
                    `usptobias.results_NA.UniquePauth`
        )
            
    LEFT JOIN (
            SELECT
                    appln_id,
                    appln_nr,
                    appln_auth,
                    appln_kind,
                    appln_filing_year,
                    docdb_family_id,
                    docdb_family_size
            FROM
                    `usptobias.patstat.tls201`
            )
            USING(appln_id)
"""

# Defining the query
query_job = client.query(query, location='US', job_config=job_config)

query_job.result()

In [46]:
# from google.cloud import bigquery
# client = bigquery.Client()
dataset_id = 'results_NA'
table_id = 'UniquePauth2'

dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)
table = client.get_table(table_ref)  # API Request

# View table properties
print(table.schema)
print(table.description)
print("\nThere are {:,} number of records in total!".format(table.num_rows))

# Load the first 10 rows
rows = client.list_rows(table, max_results=10000)

#pd.DataFrame(rows)
df_ = rows.to_dataframe()

[SchemaField('appln_id', 'INTEGER', 'NULLABLE', None, ()), SchemaField('custom_family_id', 'INTEGER', 'NULLABLE', None, ()), SchemaField('appln_nr', 'STRING', 'NULLABLE', None, ()), SchemaField('appln_auth', 'STRING', 'NULLABLE', None, ()), SchemaField('appln_kind', 'STRING', 'NULLABLE', None, ()), SchemaField('appln_filing_year', 'INTEGER', 'NULLABLE', None, ()), SchemaField('docdb_family_id', 'INTEGER', 'NULLABLE', None, ()), SchemaField('docdb_family_size', 'INTEGER', 'NULLABLE', None, ())]
None

There are 24,449,360 number of records in total!


In [47]:
df_.head()

,appln_id,custom_family_id,appln_nr,appln_auth,appln_kind,appln_filing_year,docdb_family_id,docdb_family_size
0,18412452,902171900,32264D,FI,A,1959,36896480,1
1,42222857,49581954,51038,OA,A,1964,23243220,17
2,18419321,902191434,37780D,FI,A,1961,36901969,1
3,41214687,902397788,156,MC,A,1959,8706677,5
4,41183982,18938961,46641D,LU,A,1964,8809891,5


# Creating `'USPatents'` Dataset

In [48]:
client = bigquery.Client()

# Creating Job Config
job_config = bigquery.QueryJobConfig()
#job_config.dry_run = True
job_config.use_query_cache = False
# Set configuration.query.writeDisposition
job_config.write_disposition = 'WRITE_TRUNCATE'

# Set the destination table
dataset_id = 'results_NA'
table_id = 'UsPatents'
table_ref = client.dataset(dataset_id).table(table_id)
job_config.destination = table_ref


query = """
    SELECT
            *
    FROM 
            `usptobias.results_NA.UniquePauth2`
    WHERE
            appln_auth = 'US'
            AND appln_filing_year BETWEEN 2001 AND 2018
            AND appln_kind = 'A '
     """
# Defining the query
query_job = client.query(query, location='US', job_config=job_config)

query_job.result()

In [49]:
# from google.cloud import bigquery
# client = bigquery.Client()
dataset_id = 'results_NA'
table_id = 'UsPatents'

dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)
table = client.get_table(table_ref)  # API Request

# View table properties
print(table.schema)
print(table.description)
print("\nThere are {:,} number of records in total!".format(table.num_rows))

# Load the first 10 rows
rows = client.list_rows(table, max_results=10000)

#pd.DataFrame(rows)
df_ = rows.to_dataframe()

[SchemaField('appln_id', 'INTEGER', 'NULLABLE', None, ()), SchemaField('custom_family_id', 'INTEGER', 'NULLABLE', None, ()), SchemaField('appln_nr', 'STRING', 'NULLABLE', None, ()), SchemaField('appln_auth', 'STRING', 'NULLABLE', None, ()), SchemaField('appln_kind', 'STRING', 'NULLABLE', None, ()), SchemaField('appln_filing_year', 'INTEGER', 'NULLABLE', None, ()), SchemaField('docdb_family_id', 'INTEGER', 'NULLABLE', None, ()), SchemaField('docdb_family_size', 'INTEGER', 'NULLABLE', None, ())]
None

There are 2,202,790 number of records in total!


# Joining `"UsPatents"` dataset on `"UniquePauth"`

In [50]:
client = bigquery.Client()

# Creating Job Config
job_config = bigquery.QueryJobConfig()
#job_config.dry_run = True
job_config.use_query_cache = False
# Set configuration.query.writeDisposition
job_config.write_disposition = 'WRITE_TRUNCATE'

# Set the destination table
dataset_id = 'results_NA'
table_id = 'UsJoinedCustom'
table_ref = client.dataset(dataset_id).table(table_id)
job_config.destination = table_ref


query = """
    SELECT
            t1.appln_id AS appln_id_1,
            t1.appln_nr AS appln_nr_1,
            t1.appln_auth AS appln_auth_1,
            t1.appln_filing_year AS appln_filing_year_1,
            t1.appln_kind AS appln_kind_1,
            t2.appln_id AS appln_id_2,
            t2.appln_nr AS appln_nr_2,
            t2.appln_auth AS appln_auth_2,
            t2.appln_filing_year AS appln_filing_year_2,
            t2.appln_kind AS appln_kind_2,
            t1.custom_family_id AS custom_family_id
    FROM
            `usptobias.results_NA.UsPatents` AS t1
    LEFT JOIN
            `usptobias.results_NA.UniquePauth2` AS t2
        ON
            t1.docdb_family_id=t2.docdb_family_id
    WHERE
            t2.appln_auth<>'US'
     """
# Defining the query
query_job = client.query(query, location='US', job_config=job_config)

query_job.result()

In [51]:
# Set the destination table
dataset_id = 'results_NA'
table_id = 'UsJoinedCustom'

dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)
table = client.get_table(table_ref)  # API Request

# View table properties
print(table.schema)
print(table.description)
print("\nThere are {:,} number of records in total!".format(table.num_rows))

# Load the first 10 rows
rows = client.list_rows(table, max_results=10000)

#pd.DataFrame(rows)
df_ = rows.to_dataframe()

[SchemaField('appln_id_1', 'INTEGER', 'NULLABLE', None, ()), SchemaField('appln_nr_1', 'STRING', 'NULLABLE', None, ()), SchemaField('appln_auth_1', 'STRING', 'NULLABLE', None, ()), SchemaField('appln_filing_year_1', 'INTEGER', 'NULLABLE', None, ()), SchemaField('appln_kind_1', 'STRING', 'NULLABLE', None, ()), SchemaField('appln_id_2', 'INTEGER', 'NULLABLE', None, ()), SchemaField('appln_nr_2', 'STRING', 'NULLABLE', None, ()), SchemaField('appln_auth_2', 'STRING', 'NULLABLE', None, ()), SchemaField('appln_filing_year_2', 'INTEGER', 'NULLABLE', None, ()), SchemaField('appln_kind_2', 'STRING', 'NULLABLE', None, ()), SchemaField('custom_family_id', 'INTEGER', 'NULLABLE', None, ())]
None

There are 4,685,533 number of records in total!


In [52]:
df_.head(15)

,appln_id_1,appln_nr_1,appln_auth_1,appln_filing_year_1,appln_kind_1,appln_id_2,appln_nr_2,appln_auth_2,appln_filing_year_2,appln_kind_2,custom_family_id
0,496923782,201815859849,US,2018,A,496684957,1850112,FR,2018,A,496395676
1,54402932,97576501,US,2001,A,15569269,200000013,EA,1998,A,905602187
2,51606446,53094101,US,2001,A,42551350,98954847,PT,1998,T,41485142
3,48740419,21745702,US,2002,A,44967710,9700427,TR,1997,A,13430657
4,495743444,201615576573,US,2016,A,496213491,2018000254,CO,2018,A,902777002
5,54155043,92116001,US,2001,A,23979498,9900176,HU,1996,A,23169864
6,53875924,86332907,US,2007,A,15569684,200000531,EA,1998,A,20561452
7,54392712,97324204,US,2004,A,42450558,34181698,PL,1998,A,20572668
8,53770765,84416601,US,2001,A,42437582,32839897,PL,1997,A,901325140
9,47218456,9309302,US,2002,A,380731751,50598,YU,1997,A,52125845
